<a href="https://colab.research.google.com/github/DaeSeokSong/Music-VAE/blob/master/MVAE_assignment_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

## install

In [22]:
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

print('Loading model checkpoints')
!gsutil -q -m cp gs://magentadata/models/music_vae/checkpoints/* .

Installing dependencies...
Loading model checkpoints


## Import

In [27]:
import glob

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

import note_seq

import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Importing libraries and defining some helper functions...


## Funtion

In [4]:
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
    if lib == 'fluidsynth':
        return 'libfluidsynth.so.1'
    else:
        return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

def play(note_sequence):
    mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
    """Interpolates between a start and end sequence."""
    note_sequences = model.interpolate(
        start_seq, end_seq,num_steps=num_steps, length=max_length,
        temperature=temperature,
        assert_same_length=assert_same_length)

    print('Start Seq Reconstruction')
    play(note_sequences[0])

    print('End Seq Reconstruction')
    play(note_sequences[-1])

    print('Mean Sequence')
    play(note_sequences[num_steps // 2])

    print('Start -> End Interpolation')
    interp_seq = mm.sequences_lib.concatenate_sequences(
        note_sequences, [individual_duration] * len(note_sequences))
    
    play(interp_seq)
    mm.plot_sequence(interp_seq)

    return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
    mm.sequence_proto_to_midi_file(note_sequence, filename)
    files.download(filename)

print('Done')

Done


# Extract 4-bar drum sample

## Grobal variable

In [24]:
GROOVAE_4BAR = 'groovae_4bar.tar'
CAT_DRUMS_2BAR_SMALL_HIKL = 'cat-drums_2bar_small.hikl.tar'
CAT_DRUMS_2BAR_SMALL_LOKL = 'cat-drums_2bar_small.lokl.tar'
NADE_DRUMS_2BAR_FULL = 'nade-drums_2bar_full.tar'

## Model

In [25]:
#@title Load Pretrained Models
drums_models = {}

# Groove
# 4-bar
groovae_4bar_config = configs.CONFIG_MAP['groovae_4bar']
drums_models['groovae_4bar'] = TrainedModel(groovae_4bar_config, batch_size=8, checkpoint_dir_or_path=GROOVAE_4BAR)

# One-hot encoded.
# 2-bar
drums_config = configs.CONFIG_MAP['cat-drums_2bar_small']
drums_models['drums_2bar_oh_lokl'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path=CAT_DRUMS_2BAR_SMALL_HIKL)
drums_models['drums_2bar_oh_hikl'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path=CAT_DRUMS_2BAR_SMALL_LOKL)

# Multi-label NADE.
# 2-bar
drums_nade_full_config = configs.CONFIG_MAP['nade-drums_2bar_full']
drums_models['drums_2bar_nade_full'] = TrainedModel(drums_nade_full_config, batch_size=4, checkpoint_dir_or_path=NADE_DRUMS_2BAR_FULL)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 8, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 8, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]



INFO:tensorflow:
Decoder Cells:
  units: [256, 256]



...

Instructions for updating:
Use the `axis` argument instead


INFO:tensorflow:Unbundling checkpoint.


INFO:tensorflow:Unbundling checkpoint.


INFO:tensorflow:Restoring parameters from /tmp/tmpt6yu21oq/nade-drums_2bar_full.ckpt


INFO:tensorflow:Restoring parameters from /tmp/tmpt6yu21oq/nade-drums_2bar_full.ckpt


In [31]:
#@title Generate samples as many as sample_num from the prior of one of the models listed above.

drums_sample_model = "groovae_4bar" #@param ["groovae_4bar", "drums_2bar_oh_lokl", "drums_2bar_oh_hikl", "drums_2bar_nade_full"]
temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
sample_num = 1 #@param {type:"slider", min:1, max:4, step:1}
drums_samples = drums_models[drums_sample_model].sample(n=sample_num, length=32, temperature=temperature)
for idx, ns in enumerate(drums_samples):
    print("Beat ", idx)
    note_seq.plot_sequence(ns)
    play(ns)
    print('\n\n')

Beat  0


In [21]:
#@title Optionally download generated MIDI samples.

for i, ns in enumerate(drums_samples):
  download(ns, '%s_sample_%d.mid' % (drums_sample_model, i))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Train and Generate interpolation

## Train

In [36]:
#@title upload own MIDI files to use for interpolation endpoints instead of those provided.
input_drums_midi_data = files.upload().values()

Saving 1_funk-groove1_138_beat_4-4.mid to 1_funk-groove1_138_beat_4-4 (1).mid


In [37]:
#@title Extract drums from MIDI files. This will extract all unique 2-bar drum beats using a sliding window with a stride of 1 bar.
drums_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_drums_midi_data]
extracted_beats = []

for ns in drums_input_seqs:
  extracted_beats.extend(groovae_4bar_config.data_converter.from_tensors(
      groovae_4bar_config.data_converter.to_tensors(ns)[0]))

for idx, ns in enumerate(extracted_beats):
  print("Beat", idx)
  play(ns)

Beat 0


Beat 1


ValueError: ignored